In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Step 1: Fetch historical stock price data for AAPL
data = yf.download('TSLA', start='2020-01-01', end='2025-07-05')
df = data[['Close']].copy()
df.columns = ['price']

In [ ]:
#Step 2: Calculate daily return, 20-day SMA, and standard deviation
df['daily_return'] = df['price'].pct_change()
window = 20
df['sma'] = df['price'].rolling(window=window).mean()
df['std_dev'] = df['price'].rolling(window=window).std()
df['lower_band'] = df['sma'] - (1.6 * df['std_dev']) # Threshold: 1 standard deviation below SMA

In [ ]:
#Step 3 & 4: Define trading signals and simulate positions
position = []
in_position = False

for i in range(len(df)):
    if i < window: # Skip first 20 days due to NaN in rolling calculations
        position.append(0)
        continue

    price = df['price'].iloc[i]
    sma = df['sma'].iloc[i]
    lower_band = df['lower_band'].iloc[i]

    if not in_position and price < lower_band:
        in_position = True      # Buy Signal: price below lower band
    elif in_position and price >= sma:
        in_position = False     # Sell Singal: price equal or above sma

    position.append(1 if in_position else 0)

df['position'] = position

df.head(30)

In [ ]:
# Step 5: Calculate strategy returns
# Position at close of day i-1 determines return on day i
df['strategy_return'] = df['position'].shift(1) * df['daily_return']
df['strategy_return'] = df['strategy_return'].fillna(0) # Fill NaN with 0 for days with no position

# Calculate cumulative return for strategy and buy-and-hold
df['cum_strategy_retun'] = (1 + df['strategy_return']).cumprod() - 1
df['cum_bh_return'] = (1 + df['daily_return']).cumprod() - 1

In [ ]:
# Step 6: Visualize the results
plt.figure(figsize = (10,6))
plt.plot(df.index, df['cum_strategy_retun'], label = "Mean Reversion Strategy")
plt.plot(df.index, df['cum_bh_return'], label = 'Buy and hold')
plt.title('Mean Reversion Strat vs. Buy and Hold (AAPL)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.show()

strategy_total_return = df['cum_strategy_retun'].iloc[-1]
bh_total_return = df['cum_bh_return'].iloc[-1]
strategy_sharpe = df['strategy_return'].mean() / df['strategy_return'].std() * np.sqrt(252)

print(f"Strategy Total Return: {strategy_total_return:.3%}")
print(f"Buy and Hold Total Return: {bh_total_return:.3%}")
print(f"Strategy Sharpe Ratio: {strategy_sharpe:.3f}")